In [2]:
import numpy as np
datanpy = np.load('./urban-processed/data70.npy')

In [3]:
files_list_npy = np.load('./urban-processed/files_list70.npy',allow_pickle=True)
times_list_npy = np.load('./urban-processed/times_list70.npy',allow_pickle=True)

In [4]:
'''
newdata = []
for i in range(len(datanpy)):
    if i%10 ==0:
        print(i)
    for j in range(len(datanpy[i])):
        for k in range(len(datanpy[i][j])):
            newdata.append((datanpy[i][j][k]+datanpy[i][j][k])/2)
'''
datanpy =datanpy[:,0:100,0:100]

In [5]:
print(np.shape(datanpy))
datanpy

(8732, 70, 70)


array([[[6.79378591e-13, 3.17790017e-10, 1.14418364e-10, ...,
         1.16217524e-08, 7.54488916e-10, 5.26129673e-09],
        [1.68355798e-11, 3.55363267e-10, 8.35323338e-11, ...,
         3.86343046e-09, 4.37743231e-10, 2.11693263e-09],
        [6.26813115e-11, 1.08135312e-09, 6.54149845e-10, ...,
         1.24940192e-09, 3.76949028e-10, 4.86793161e-10],
        ...,
        [8.06595730e-14, 1.93347251e-13, 2.15119229e-08, ...,
         4.39520989e-11, 5.42613567e-12, 2.30496299e-11],
        [5.58384888e-14, 1.81193656e-13, 1.09083551e-08, ...,
         3.99611802e-11, 2.92926208e-12, 1.71715558e-11],
        [2.04783727e-13, 2.83981941e-14, 6.74947254e-09, ...,
         5.43933605e-12, 6.09059764e-12, 1.85631684e-11]],

       [[5.69664038e-10, 1.63528580e-10, 1.36504461e-10, ...,
         3.65890457e-10, 1.62544242e-10, 3.52759988e-10],
        [9.26960106e-11, 4.55008670e-10, 3.78689163e-10, ...,
         6.10788989e-11, 7.88580520e-11, 6.17655649e-11],
        [4.82548224e-10, 

In [6]:
import pandas as pd
urban = pd.read_csv(r"./urban-sounds/UrbanSound8K.csv")
set(urban['class'].tolist())

{'air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music'}

In [7]:
classIdNpy = []
for i in range(len(files_list_npy)):
    if i%1000 == 0:
        print(i)
    classIdNpy.append(urban[urban['slice_file_name']==files_list_npy[i]]['classID'].tolist()[0])
    

0
1000
2000
3000
4000
5000
6000
7000
8000


In [8]:
from __future__ import print_function
import keras,sklearn
# suppress tensorflow compilation warnings
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
seed=0
np.random.seed(seed) # fix random seed
tf.random.set_seed(seed)
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [10]:
dataset_size = len(datanpy)
train_size = int(.8*dataset_size)
img_rows, img_cols = 70,70

num_classes = 10

X = datanpy
Y = classIdNpy
files_list = files_list_npy

X_train = X[:train_size]
Y_train = Y[:train_size]

X_test = X[train_size:dataset_size]
Y_test = Y[train_size:dataset_size]

X_train = X_train.reshape(X_train.shape[0], img_rows*img_cols)
X_test = X_test.reshape(X_test.shape[0], img_rows*img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= X_train.max()
X_test /= X_test.max()

Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test,num_classes)

In [11]:
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (6985, 4900)
Y_train shape: (6985, 10)

6985 train samples
1747 test samples


In [12]:
# reshape data, depending on Keras backend
if keras.backend.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (6985, 70, 70, 1)
Y_train shape: (6985, 10)

6985 train samples
1747 test samples


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

def create_CNN():
    # instantiate model
    model = Sequential()
    # add first convolutional layer with 10 filters (dimensionality of output space)
    model.add(Conv2D(3, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))
    # add 2D pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # add second convolutional layer with 20 filters
    model.add(Conv2D(4, (6, 6), activation='relu'))
    # apply dropout with rate 0.5
    model.add(Dropout(0.5))
    # add 2D pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # flatten data
    model.add(Flatten())
    # add a dense all-to-all relu layer
    model.add(Dense(300,input_dim=4*14*14, activation='relu'))
    # apply dropout with rate 0.5
    model.add(Dropout(0.5))
    
    model.add(Dense(100,input_dim=300, activation='relu'))
    # apply dropout with rate 0.5
    model.add(Dropout(0.5))
    # soft-max layer
    model.add(Dense(num_classes, input_dim=100, activation='softmax'))
    
    # compile the model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='Adam',
                  metrics=['accuracy'])
    
    return model

In [19]:
# training parameters
batch_size = 64
epochs = 75

# create the deep conv net
print('creating CNN')
model_CNN=create_CNN()
print('done creating CNN!')

print('train begin...')
# train CNN

model_CNN.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

# evaliate model
score = model_CNN.evaluate(X_test, Y_test, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

creating CNN
done creating CNN!
train begin...
Train on 6985 samples, validate on 1747 samples
Epoch 1/75
6985/6985 [==============================] - 7s 1ms/step - loss: 2.2772 - accuracy: 0.1150 - val_loss: 2.2417 - val_accuracy: 0.1448
Epoch 2/75
6985/6985 [==============================] - 8s 1ms/step - loss: 2.2217 - accuracy: 0.1605 - val_loss: 2.1726 - val_accuracy: 0.2221
Epoch 3/75
6985/6985 [==============================] - 8s 1ms/step - loss: 2.1326 - accuracy: 0.2052 - val_loss: 2.0255 - val_accuracy: 0.3148
Epoch 4/75
6985/6985 [==============================] - 8s 1ms/step - loss: 2.0426 - accuracy: 0.2503 - val_loss: 2.0283 - val_accuracy: 0.2931
Epoch 5/75
6985/6985 [==============================] - 8s 1ms/step - loss: 1.9862 - accuracy: 0.2697 - val_loss: 1.9931 - val_accuracy: 0.3091
Epoch 6/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.9516 - accuracy: 0.2862 - val_loss: 1.9329 - val_accuracy: 0.3188
Epoch 7/75
6985/6985 [===================

Epoch 57/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.1572 - accuracy: 0.5960 - val_loss: 2.2493 - val_accuracy: 0.4505
Epoch 58/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.1723 - accuracy: 0.6046 - val_loss: 2.1469 - val_accuracy: 0.5072
Epoch 59/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.2000 - accuracy: 0.5825 - val_loss: 2.2985 - val_accuracy: 0.4390
Epoch 60/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.1836 - accuracy: 0.5865 - val_loss: 2.2671 - val_accuracy: 0.4762
Epoch 61/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.1640 - accuracy: 0.6033 - val_loss: 2.2792 - val_accuracy: 0.4591
Epoch 62/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.1387 - accuracy: 0.5948 - val_loss: 2.3406 - val_accuracy: 0.4591
Epoch 63/75
6985/6985 [==============================] - 9s 1ms/step - loss: 1.1156 - accuracy: 0.6109 - val_loss: 2.2987 - val_ac

In [18]:
score = model_CNN.evaluate(X_test, Y_test, verbose=1)

print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

1747/1747 [==============================] - 1s 328us/step

Test loss: 2.116617216747286
Test accuracy: 0.45392102003097534
